# **Dataset**

This colab notebook is for the purpose of uploading csv files, inspecting/transforming/concatenating them and for finding the Trigger.

In [ ]:
#Import dependencies

import pandas as pd               #Python Data Analysis Library
import matplotlib.pyplot as plt   #graphing library
import numpy as np
import csv
import datetime

In [ ]:
#Upload dataset/s
from google.colab import files

answer_var = input('Lokale Datei/n hochladen? (j/n): ')

if answer_var == 'j':
  uploaded = files.upload()
  print('Hochladen abgeschlossen.')
elif answer_var == 'n':
  # wenn benötigt einfügen
  print('Github / Gmail folder upload noch nicht implementiert.')
else:
  print('Falsche Eingabe!')
  

# Inspect uploaded dataset/s

In [ ]:
filenames = uploaded.keys()   #Dictionary of all uploaded file names
column_name = 'Time (us)'     #'Time'
column_name2 = 'Voltage (V)'  #'Channel A'                                  
filenameiterator = 0        #holds helper var
g_nan_counter = 0
g_zero_counter = 0

answer_var = input('Alle hochgeladenen Dateien inspizieren? (j/n): ')

def inspect_file(fn_var):
  df_inspect = pd.read_csv(fn_var, sep=',', decimal='.',header = None, skiprows = 3, names = [column_name,column_name2]) #sep und decimal anpassung nicht vergessen
  
  #Lokal die Anzahl der 0 Werte und fehlender Werte bestimmen und zähler erhöhen, sofern nötig
  nan_counter = df_inspect[column_name2].isna().sum()
  global g_nan_counter
  g_nan_counter = g_nan_counter + nan_counter
  try:                                                        
    zero_counter = df_inspect[column_name2].value_counts(dropna=False)[0]
    global g_zero_counter
    g_zero_counter = g_zero_counter + zero_counter               
  except KeyError:
    zero_counter = 0

  #Graph Zeichnen
  plt.figure(figsize=(20,5),dpi=150)
  plt.suptitle(fn_var + ' Anzahl 0 Werte: ' + str(zero_counter) +' ,Anzahl f Werte: ' + str(nan_counter))
  plt.plot(df_inspect[column_name2])
  plt.xlabel('Samples')
  plt.ylabel('Spannung in V')                                 #Osszi trigger bei den Werten Beachten
  plt.show()

#Entweder alle hochgeladenen Dateien oder nur eine spezifische inspizieren
if answer_var == 'j':
  for fn in uploaded.keys():                                    
    inspected_filename = list(filenames)[filenameiterator]      
    inspect_file(inspected_filename)
    filenameiterator=filenameiterator+1
elif answer_var == 'n':
  #.csv am ende nicht vergessen z.B. '20220520-0001.csv'
  answer_var2 = input('Bitte spezifische Datei zum inspizieren angeben: ')  
  try:
    inspect_file(answer_var2)
  except FileNotFoundError:
    print('Die Datei ' + answer_var2 + ' konnte nicht gefunden werden.')
else:
  print('Falsche Eingabe!')

#Gesamtmenge aller 0 und fehler ausgeben
print('Gesamtmenge an 0 Werten: '+ str(g_zero_counter)+' , Gesamtmenge fehlender Werte: '+ str(g_nan_counter))





# Clean uploaded dataset/s

In [ ]:
#Bereinigen von 0 Werten, fehlenden Werten und ausreißern. Seperates speichern in C_filname.csv
#Anpassbar durch folgende bools
null_values = True
missing_values = True
outlier_values = True
min_outlier_size = 0.0001    #how many lowest quantiles to take
max_outlier_size = 0.9999    #how many highest quantiles to take

filenames = uploaded.keys() #Dictionary of all uploaded file names                                   
filenameiterator = 0        #holds helper var
g_cleaned_counter = 0
g_nan_counter = 0
g_zero_counter = 0
g_outlier_counter = 0

answer_var = input('Alle hochgeladenen Dateien bereinigen? (j/n): ')

def clean_file(fn_var):
  df_clean = pd.read_csv(fn_var, sep=';', decimal=',',header = None, skiprows = 3, names = ['Time','Channel A']) #sep und decimal anpassung nicht vergessen
  
  #Anzahl der fehlenden Werte bestimmen und beheben
  if null_values == True:
    nan_counter = df_clean['Channel A'].isna().sum()
    df_clean['Channel A'].fillna(0, inplace=True)
    global g_nan_counter
    g_nan_counter = g_nan_counter + nan_counter

  #Anzahl der 0 Werte bestimmen und beheben
  if missing_values == True:
    try:                                                        
      zero_counter = df_clean['Channel A'].value_counts(dropna=False)[0]
      global g_zero_counter
      g_zero_counter = g_zero_counter + zero_counter
      for z in range(df_clean['Channel A'].size):
        if df_clean.at[z,'Channel A'] == 0:
          df_clean.at[z,'Channel A'] = df_clean.at[z-1,'Channel A']
    except KeyError:
      zero_counter = 0

  #Anzahl der ausreißer bestimmen und beheben
  if outlier_values == True:
    outlier_counter = 0
    min_threshold, max_threshold = df_clean['Channel A'].quantile([min_outlier_size,max_outlier_size])
    #print(df_clean[df_clean['Channel A']>max_threshold])
    #print(df_clean[df_clean['Channel A']<min_threshold])
    for y in range(df_clean['Channel A'].size):
      if df_clean.at[y,'Channel A'] <min_threshold:
        df_clean.at[y,'Channel A'] = df_clean.at[y-1,'Channel A']
        outlier_counter = outlier_counter + 1
      if df_clean.at[y,'Channel A'] >max_threshold:
        df_clean.at[y,'Channel A'] = df_clean.at[y-1,'Channel A']
        outlier_counter = outlier_counter + 1
    global g_outlier_counter
    g_outlier_counter = g_outlier_counter + outlier_counter

  #Graph Zeichnen
  plt.figure(figsize=(20,5),dpi=150)
  plt.suptitle(fn_var + ' Null Behoben: ' + str(zero_counter) +' ,Fehlend Behoben: ' + str(nan_counter) + ' ,Ausreißer Behoben: ' + str(outlier_counter))
  plt.plot(df_clean['Channel A'])
  plt.xlabel('Samples')
  plt.ylabel('Spannung in V')                                 #Osszi trigger bei den Werten Beachten
  plt.show()

  #Graph als seperate csv speichern
  f = open('C_' + fn_var, 'w')                     
  f.close()
  df_clean['Channel A'].to_csv('C_' + fn_var, index=False)


#Entweder alle hochgeladenen Dateien oder nur eine spezifische bereinigen
if answer_var == 'j':
  for fn in uploaded.keys():                                    
    inspected_filename = list(filenames)[filenameiterator]      
    clean_file(inspected_filename)
    filenameiterator=filenameiterator+1
elif answer_var == 'n':
  #.csv am ende nicht vergessen z.B. '20220520-0001.csv'
  answer_var2 = input('Bitte spezifische Datei zum bereinigen angeben: ')  
  try:
    clean_file(answer_var2)
  except FileNotFoundError:
    print('Die Datei ' + answer_var2 + ' konnte nicht gefunden werden.')
else:
  print('Falsche Eingabe!')

#Gesamtmenge aller 0 und fehler ausgeben
g_cleaned_counter = g_nan_counter + g_outlier_counter + g_nan_counter
print('Insgesamt wurden '+ str(g_cleaned_counter)+' Samples bereinigt.')


# Find the first trigger and save as the full frame in columns

In [ ]:
#Find the first trigger in every uploaded csv and delete all preceding samples. Then save the shortend frames as columns in Full_dataframe_output + date.csv
use_cleaned_files = False                        #should the cleaned files be used for finding the trigger
smoothingamount = 5                             #how many samples are used to smooth the triggerfinder
column_name = 'Time (us)'     #'Time'
column_name2 = 'Voltage (V)'  #'Channel A'
column_seperator = ','        #';'
column_decimal = '.'          #',' 
triggerfall_threshold = 15                      #threshold to find falling trigger in % downturn
max_trigger_search_range = 5000                 #how many samples should be checked for the trigger
output_filename = '301-500_Full_dataframe_113'

filenames = uploaded.keys()                     #Dictionary of all uploaded file names
filenameiterator = 0                            #holds helper var
thresholdvar = 0                                #holds helper var

f = open(str(output_filename + '.csv'), 'w')                     
f.close()

for fn in uploaded.keys():                                  #for schleife auskommentieren wenn nicht alle bearbeitet werden sollen          
  if use_cleaned_files == False:
    str_filename = list(filenames)[filenameiterator]
    df_smooth = pd.read_csv(str_filename, sep=column_seperator, decimal=column_decimal,header = None, skiprows = 3, names = [column_name,column_name2]) #sep und decimal anpassung nicht vergessen
    df_triggerframe = pd.read_csv(str_filename, sep=column_seperator, decimal=column_decimal,header = None, skiprows = 3, names = [column_name,column_name2])  
  else:
    str_filename = 'C_'+list(filenames)[filenameiterator]
    df_smooth = pd.read_csv(str_filename, sep=column_seperator, decimal=column_decimal,header = None, skiprows = 1, names = [column_name2]) #sep und decimal . statt komma
    df_triggerframe = pd.read_csv(str_filename, sep=column_seperator, decimal=column_decimal,header = None, skiprows = 1, names = [column_name2]) 

  for y in range(max_trigger_search_range):                                               #Trigger in search range suchen, zeilen löschen die geringer als ave_low_voltage sind
    
    for z in range(smoothingamount):
      thresholdvar = thresholdvar + df_triggerframe.at[y+z,column_name2]

    thresholdvar = df_triggerframe.at[y,column_name2]-(thresholdvar/smoothingamount)
    #print(thresholdvar)
    if thresholdvar/df_triggerframe.at[y,column_name2]<1/triggerfall_threshold:
      df_triggerframe.drop([y],inplace = True)
    else:
      df_triggerframe.reset_index(drop=True, inplace=True)                                #reassign index to triggerframe
      break

  plt.figure(figsize=(20,5),dpi=150)
  plt.plot(df_triggerframe[column_name2])
  plt.xlabel('Samples')
  plt.ylabel('Spannung in V')                                                             #Osszi trigger bei den Werten Beachten
  plt.show()

  if filenameiterator==0:                                                                 #Daten der ersten csv im output sichern  
    df_triggerframe[column_name2].to_csv(str(output_filename + '.csv'), index=False)
  else:                                                                                   #Daten der folgenden csv's an die output csv als neue spalten anhängen 
    csv_append = pd.read_csv(str(output_filename + '.csv'))
    csv_append.insert(filenameiterator, column_name2+' Aufnahme:'+str(filenameiterator+1), df_triggerframe[column_name2], True)
    csv_append.to_csv(str(output_filename + '.csv'), index=False)

  thresholdvar = 0
  filenameiterator=filenameiterator+1                                                     #iterator zum lesen der nächste csv erhöhen


#Find all 0 bits and save as columns

In [ ]:
#Find all 0 bits, Then save them as columns in 0_bit_output + date.csv
bit_size = 249                #how many samples represent a bit
bit_difference_size = 7       #how far apart should min and max values of a frame be to count as 0 or 1
column_name = 'Time (us)'     #'Time'
column_name2 = 'Voltage (V)'  #'Channel A'
column_seperator = ','        #';'
column_decimal = '.'          #','
zero_bits_per_file = 100      #how many zero bits should be in a file before a new one starts
increased_bit = 34.6          #ab welchem wert zählt ein 0 bit als gesteigert, setting nicht vergessen!

output_filename = '113_0_bit_output_mx_(301-500)_'
input_filename = '301-500_Full_dataframe_113' + '.csv'  

#holds helper var                          
first_append = True
filecounter = 0
sample_counter = 0
total_bit_counter = 0

#load input file
df_input = pd.read_csv(input_filename, sep=column_seperator, decimal=column_decimal)

#go through each coloumn of the input file
for fn in range(len(df_input.columns)):
  str_column_name = list(df_input)[fn]
  df_0_bit_finder = pd.DataFrame(data=df_input[str_column_name], columns=[str_column_name])
  df_bit_size_frame = pd.DataFrame(index=range(bit_size),columns=[str_column_name])
  #go through a row and save as new column when bit_size is reached
  for fs in range(df_0_bit_finder[str_column_name].size):
    df_bit_size_frame.at[sample_counter,str_column_name] = df_0_bit_finder.at[fs,str_column_name]
    sample_counter = sample_counter + 1
    #if sample size is reached, start writing in a column
    if sample_counter > bit_size:
      #get min and max values to differentiate between 1 and 0 bits
      minvalue = df_bit_size_frame[str_column_name].min()
      maxvalue = df_bit_size_frame[str_column_name].max()
      if maxvalue-minvalue > bit_difference_size:

        #WICHTIG!!!
        #if maxvalue < increased_bit:  #sollen nur sich nicht steigernde 0 bits gespeichert werden?
        if maxvalue > increased_bit: #sollen nur gesteigerte 0 bits gespeichert werden?
        #SETTING!!! beide auskommentieren für mixed bits oder EINES an je nachdem was gespeichert werden soll

          if first_append == True:
            f = open(output_filename + str(filecounter) + '.csv', 'w')                    
            f.close()
            df_bit_size_frame.to_csv(output_filename + str(filecounter) + '.csv', index=False)
            first_append = False
          else:
            csv_append = pd.read_csv(output_filename + str(filecounter) + '.csv')
            csv_append.insert(total_bit_counter,str_column_name+' bit:'+ str(total_bit_counter), df_bit_size_frame[str_column_name], True)
            csv_append.to_csv(output_filename + str(filecounter) + '.csv', index=False)
            sample_counter = 0
            total_bit_counter = total_bit_counter+1
            #initiate writing of new file in the next loop if zero bit per file is reached
            if total_bit_counter == zero_bits_per_file-1:
              sample_counter = 0
              filecounter = filecounter+1
              total_bit_counter = 0
              first_append = True
              print('file number:'+ str(filecounter)+' is done.')

        else:
          sample_counter = 0

      else:
        sample_counter = 0

  #reset helper var for next loop
  sample_counter = 0 